# I go through the jax tutorial and attempt to understand 10% of what goes on

In [1]:
import jax.numpy as jnp
from jax import grad
import jax

# Key Concepts: 
- jax.Array Creation: Similar to np.  
- There be some complicated stuff about devices that ill get to later
- Tracers: Basically u run through fxn with tracer and they help jax "compile"/figure out the sequence of operations the fxn carries out (aka the jaxpr)
- Pytree: Nested data structures
- Jax has random keys that you pass in to fxns instead of np's seed

# JIT


In [18]:
global_list = []

def log2(x, k):
  global_list.append(x)
  ln_x = jnp.log(x)
  ln_2 = jnp.log(k)
  return ln_x / ln_2

print(jax.make_jaxpr(log2)(3.0, 5.)) 
#Jaxpr: Low-level, compiled thing. You don't run it. It just exists (kind of like backend stuff thats still cool)
#jaxpr made by running it on the args u give while also tracing stuff.

{ lambda ; a:f32[] b:f32[]. let
    c:f32[] = log a
    d:f32[] = log b
    e:f32[] = div c d
  in (e,) }


# IMPORTANT: Does not capture anything about global_list.append(x)
- Feature, not a bug. Basically wants fxns that depend only on their args
- Impure fxns (fxns that read/write to a global state are bad due to the compiler doing weird things).
- Ie it can cache the global state as 4.0, then the global gets updated to 5, but the cached value stays 4 and then your computations go no no.
- Print is included as impure.
- Basically just pass anything that the fxn depends on as an arg.

In [23]:
jitted_log2 = jax.jit(log2)
jitted_log2(3, 5)
print(f"length {len(global_list)} and list {global_list} before")
jitted_log2(1, 5)
#Should make global_list bigger but it DOESNT due to thing above.
print(f"length {len(global_list)} and list {global_list} after")

#Side note: The first time does make it run the append but that might be just due the first call including a tracing pass (which includes the global append) but subsequent ones don't
#IN OTHER WORDS DONT READ/WRITE GLOBALS. 

length 2 and list [Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace>, Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace>] before
length 2 and list [Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace>, Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace>] after
